In [11]:
from typing import Tuple
import numpy as np
import deepgp
import matplotlib.pyplot as plt
from datetime import datetime
import os

import GPy
from GPy.models import GPRegression
from emukit.test_functions import forrester_function
from emukit.core.initial_designs import RandomDesign
from emukit.model_wrappers import GPyModelWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement, NegativeLowerConfidenceBound, ProbabilityOfImprovement, EntropySearch
from emukit.core.optimization import GradientAcquisitionOptimizer
from emukit.core.initial_designs import RandomDesign
from emukit.core import ParameterSpace, ContinuousParameter
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity
from gpflow.kernels import RBF, White, Linear
from tqdm import tqdm

from simulator import MainSimulator, TinySimulator
from world import DebugInfo
from pprint import pprint

main_simulator = MainSimulator()

In [2]:
def plot_prediction(X,Y,x_plot,mu_plot,var_plot,axis):
    axis.plot(X, Y, "ro", markersize=10, label="Observations")
    axis.plot(x_plot[:, 0], mu_plot[:, 0], "C0", label="Model")
    axis.fill_between(x_plot[:, 0],
                     mu_plot[:, 0] + np.sqrt(var_plot)[:, 0],
                     mu_plot[:, 0] - np.sqrt(var_plot)[:, 0], color="C0", alpha=0.6)
    axis.fill_between(x_plot[:, 0],
                     mu_plot[:, 0] + 2 * np.sqrt(var_plot)[:, 0],
                     mu_plot[:, 0] - 2 * np.sqrt(var_plot)[:, 0], color="C0", alpha=0.4)
    axis.fill_between(x_plot[:, 0],
                     mu_plot[:, 0] + 3 * np.sqrt(var_plot)[:, 0],
                     mu_plot[:, 0] - 3 * np.sqrt(var_plot)[:, 0], color="C0", alpha=0.2)
    axis.legend(loc=2, prop={'size': 10})
    axis.set(xlabel=r"$x$", ylabel=r"$f(x)$")
    axis.grid(True)

In [3]:
def plot_acquisition_functions(x_plot, ac_plots, x_new, axis):
    for ac_plot in ac_plots:
        axis.plot(x_plot, (ac_plot[1] - np.min(ac_plot[1])) / (np.max(ac_plot[1]) - np.min(ac_plot[1])), label=ac_plot[0])
    
    axis.axvline(x_new, color="red", label="x_next", linestyle="--")
    axis.legend(loc=1, prop={'size': 10})
    axis.set(xlabel=r"$x$", ylabel=r"$f(x)$")
    axis.grid(True)

In [4]:
def target_function_list(X, repetition=10):
    Y = []
    for x in X:
        mutation_rates = {
            "size": x[0],
            "speed": x[1],
            "vision": x[2],
            "aggression": x[3]
        }
        total = 0
        for i in range(repetition):
            days_survived, log = main_simulator.run(mutation_rates, debug_info=DebugInfo(
                period=10, should_display_day=False, should_display_grid=False, should_display_traits=False), max_days=10000)
            total = total + days_survived
        Y.append(total//repetition)
    return np.array(Y)[:,np.newaxis]

In [ ]:
space = ParameterSpace([ContinuousParameter('size', 0, 1),
                        ContinuousParameter('speed', 0, 1),
                        ContinuousParameter('vision', 0, 1),
                        ContinuousParameter('aggression', 0, 1)])
design = RandomDesign(space) # Collect random points
num_data_points = 5
X = design.get_samples(num_data_points)
Y = target_function_list(X)

In [8]:
kernel_rbf = GPy.kern.RBF(4, lengthscale=0.05, variance=100)
model_gpy = GPRegression(X,Y,kernel_rbf) # Train and wrap the model in Emukit
model_emukit = GPyModelWrapper(model_gpy)

In [12]:
ei_acquisition = ExpectedImprovement(model = model_emukit)
nlcb_acquisition = NegativeLowerConfidenceBound(model = model_emukit)
pi_acquisition = ProbabilityOfImprovement(model = model_emukit)
es_acquisition = EntropySearch(model = model_emukit, space = space)

In [ ]:
iterations = 1000
# The other mutation rates for the projection
projection = 0
# Along which trait is the function plotted: 0-size, 1-speed, 2-vision, 3-aggression
plot = 0

file_path = 'training_logs/plot_' + str(plot) + '_projection_' + str(projection) + '_' + str(datetime.now())
os.makedirs(file_path)
figure, axis = plt.subplots(1, 2, figsize=(20, 6))
x_plot = np.linspace(0, 1, 1000)[:, None]
x_paddings = np.linspace(projection, projection, 1000)[:, None]
x_linear = np.linspace(0, 1, 1000)[:, None]

for i in tqdm(range(iterations)):
    for j in range(plot):
        x_plot = np.append(x_paddings, x_plot, axis = 1)
    for j in range(3-plot):
        x_plot = np.append(x_plot, x_paddings, axis = 1)
        
    mu_plot, var_plot = model_emukit.predict(x_plot)
    plot_prediction(X[:,plot],Y,x_linear,mu_plot,var_plot,axis[0])
    
    ei_plot = ("EI",ei_acquisition.evaluate(x_plot))
    nlcb_plot = ("NLCB",nlcb_acquisition.evaluate(x_plot))
    pi_plot = ("PI",pi_acquisition.evaluate(x_plot))
    es_plot = ("ES",es_acquisition.evaluate(x_plot))
    
    optimizer = GradientAcquisitionOptimizer(ParameterSpace([ContinuousParameter('size', 0, 1),
                                                             ContinuousParameter('speed', 0, 1),
                                                             ContinuousParameter('vision', 0, 1),
                                                             ContinuousParameter('aggression', 0, 1)]))
    x_new, _ = optimizer.optimize(es_acquisition)
    plot_acquisition_functions(x_linear, [ei_plot,nlcb_plot,pi_plot,es_plot], x_new[0][plot], axis[1])
    print("Next position to query:", x_new)
    
    y_new = target_function_list(x_new)
    X = np.append(X, x_new, axis=0)
    Y = np.append(Y, y_new, axis=0)
    model_emukit.set_data(X, Y)
    plt.savefig(file_path + '/' + str(i) +'.png')
    plt.clf()
    figure, axis = plt.subplots(1, 2, figsize=(20, 6))

  0%|                                                                                                                                                                                                                                                                                              | 0/1000 [00:00<?, ?it/s]

In [ ]:
def sensitivity_2d(X,sensitivity):
    results = []
    for x in X:
        results.append(sensitivity.compute_statistics(np.array([x[0],x[1],projection,projection]))[1])
    return np.array(results)

In [ ]:
sensitivity = MonteCarloSensitivity(model = model_emukit, input_domain = space)
main_effects, total_effects, _ = sensitivity.compute_effects(num_monte_carlo_points = 10000)

In [ ]:
x_axis = np.linspace(0, 10, 100)
y_axis = np.linspace(0, 10, 100)

X_axis,Y_axis = np.meshgrid(x_axis, y_axis)
data = np.vstack([X_axis.ravel(), Y_axis.ravel()])
data = np.swapaxes(data,0,-1)

plt.figure(figsize=(8,6))
plt.contourf(X_axis, Y_axis, sensitivity_2d(data,sensitivity).reshape(X_axis.shape), levels=20)
plt.colorbar();